source: https://kb.objectrocket.com/postgresql/get-the-column-names-from-a-postgresql-table-with-the-psycopg2-python-adapter-756

In [6]:
%reload_ext autoreload

%autoreload 2

import os
import sys
sys.path.append('../')


import funcs 
import load_data
import mlflow
import numpy as np
import pandas as pd
import subprocess
from time import time
import git 
from psycopg2 import sql, connect
from IPython.display import display

%reload_ext load_data
%reload_ext funcs
import warnings
warnings.filterwarnings('ignore')

## ssh tunneling

In [7]:

command = 'ssh -N -L 5000:localhost:5432 artinmajdi@data7-db1.cyverse.org &'
ssh_session = subprocess.Popen('exec ' + command, stdout=subprocess.PIPE, shell=True)

## Connecting to the server

In [8]:
db_name = "chest_db"

try:
    # declare a new PostgreSQL connection object
    conn = connect(
        dbname = db_name,
        user = "artinmajdi",
        host = "localhost",
        password = "1234",
        port=5000
    )

    # print the connection if successful
    print ("psycopg2 connection:", conn)

except Exception as err:
    print ("psycopg2 connect() ERROR:", err)
    conn = None


psycopg2 connection: <connection object at 0x7fdfccc88630; dsn: 'user=artinmajdi password=xxx dbname=chest_db host=localhost port=5000', closed: 0>


## Execute the SQL object using the psycopg2 cursor object

The following segment of code passes the psycopg2.sql.SQL object to the cursor’s execute() method. This stores the data inside of the cursor. Then, the cursor’s fetchall() method call is used to have it return a list of the column names:

In [9]:
# define a function that gets the column names from a PostgreSQL table
def get_columns_names(table):

    # declare an empty list for the column names
    columns = []

    # declare cursor objects from the connection    
    col_cursor = conn.cursor()

    # concatenate string for query to get column names
    # SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'some_table';
    col_names_str = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE "
    col_names_str += "table_name = '{}';".format( table )

    # print the SQL string
    print ("\ncol_names_str:", col_names_str)

    try:
        sql_object = sql.SQL(
            # pass SQL statement to sql.SQL() method
            col_names_str
        ).format(
            # pass the identifier to the Identifier() method
            sql.Identifier( table )
        )

        # execute the SQL string to get list with col names in a tuple
        col_cursor.execute( sql_object )

        # get the tuple element from the liast
        col_names = ( col_cursor.fetchall() )

        # print list of tuples with column names
        print ("\ncol_names:", col_names)

        # iterate list of tuples and grab first element
        for tup in col_names:

            # append the col name string to the list
            columns += [ tup[0] ]

        # close the cursor object to prevent memory leaks
        col_cursor.close()

    except Exception as err:
        print ("get_columns_names ERROR:", err)

    # return the list of column names
    return columns

In [24]:
table = 'parameters'

# if the connection to PostgreSQL is valid
if conn != None:

    # pass a PostgreSQL string for the table name to the function
    columns = get_columns_names(table)

    display( pd.DataFrame(columns) )


col_names_str: SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'parameters';

col_names: [('specific_catalog',), ('specific_schema',), ('specific_name',), ('ordinal_position',), ('parameter_mode',), ('is_result',), ('as_locator',), ('parameter_name',), ('data_type',), ('character_maximum_length',), ('character_octet_length',), ('character_set_catalog',), ('character_set_schema',), ('character_set_name',), ('collation_catalog',), ('collation_schema',), ('collation_name',), ('numeric_precision',), ('numeric_precision_radix',), ('numeric_scale',), ('datetime_precision',), ('interval_type',), ('interval_precision',), ('udt_catalog',), ('udt_schema',), ('udt_name',), ('scope_catalog',), ('scope_schema',), ('scope_name',), ('maximum_cardinality',), ('dtd_identifier',), ('parameter_default',)]


,0
0,specific_catalog
1,specific_schema
2,specific_name
3,ordinal_position
4,parameter_mode
5,is_result
6,as_locator
7,parameter_name
8,data_type
9,character_maximum_length


## closing the ssh session

In [ ]:
ssh_session.kill()